In [2]:
import pandas as pd


# #write script for processing

In [3]:
df = pd.read_csv("Salinity_IMG_KO_data_compare_3.30.17.csv")
df1 = df.loc[:,["ITS_PID","Genome Name / Sample Name","path"]]
ITS_PID = list(df1["ITS_PID"])
Sample_Name = list(df1["Genome Name / Sample Name"])
path = list(df1["path"])

extractCH4 = "extract_CH4_hmm"

for a,b,c in zip(ITS_PID,Sample_Name,path):
    fa_file = str(a) + "_" + str(b) + ".fasta"
    orf_file = str(a) + "_" + str(b) + "_orfs.fasta"
    hmm_CH4output = str(a) + "_" + str(b) + "_" +extractCH4 + ".tab"
    outfilename = "run_" + str(a) + "_" + str(b) + "_"+ extractCH4 +".sh"
    outfile = open(outfilename, "w")
    outfile.write("gunzip -c " + c + " | awk '{if(NR%4==1) {printf(\">%s\\n\",substr($0,2));} else if(NR%4==2) print;}' > " + fa_file + "\n")
    outfile.write("shifter --image=kduffy/prodigal:latest --volume=/global/projectb/scratch/jzz0026/wyatt_wetland/hmmsearch:/data -- /bin/bash -c 'cd /data; prodigal -i " + fa_file + " -a " + orf_file + " -p meta'\n")
    outfile.write("rm " + fa_file + "\n")
    outfile.write("module load hmmer/3.1b2\n")
    outfile.write("hmmsearch --cpu 4 --tblout " + hmm_CH4output + " --cut_tc --noali /global/projectb/scratch/jzz0026/wyatt_wetland/"+extractCH4+".hmm " + orf_file +"\n")
    #outfile.write("rm " + orf_file + "\n")
    outfile.close()


# # write the script to submit

In [7]:
names = !ls run_*.sh
submit = open("submit.sh","w")
for each in names[1:]:
    submit.write("sbatch -t 2880 --mincpus=2 --mem=12G -D $PWD -J " + each.split("_")[1] + " --wrap=\"bash " + each +"\"\n")

submit.close()

    

# # write the script to extract reads with hmm hits

In [9]:
names = !ls *.tab
extract_query = open("extract_query.sh","w")
for each in names[1:]:
    extract_query.write("cat " + each + " | awk -F \" \" '{print $1}' | sed 's/_.*$//g' | uniq | sed '$d' > " + each.split(".")[0] + ".querylist\n")

extract_query.close()

